<a href="https://colab.research.google.com/github/kartikrupal/deep_learning/blob/main/p10_autoencoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense

# Load and preprocess the MNIST dataset
(x_train, _), (x_test, _) = mnist.load_data()
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

# Flatten the images (28x28 -> 784)
x_train = x_train.reshape((x_train.shape[0], -1))
x_test = x_test.reshape((x_test.shape[0], -1))

# Define input and encoding dimensions
input_dim = 784
encoding_dim = 32

# Build the Autoencoder architecture
# Encoder
input_layer = Input(shape=(input_dim,))
encoded = Dense(encoding_dim, activation='relu')(input_layer)

# Decoder
decoded = Dense(input_dim, activation='sigmoid')(encoded)

# Autoencoder model
autoencoder = Model(inputs=input_layer, outputs=decoded)

# Compile the model
autoencoder.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
autoencoder.fit(x_train, x_train,
                epochs=50,
                batch_size=64,
                shuffle=True,
                validation_data=(x_test, x_test))

# Evaluate on test data
loss = autoencoder.evaluate(x_test, x_test)
print("Final loss on the test data:", loss)

# Encode and decode images
encoder_model = Model(inputs=input_layer, outputs=encoded)
encoded_imgs = encoder_model.predict(x_test)

# For decoding, define a separate decoder model
encoded_input = Input(shape=(encoding_dim,))
decoder_layer = autoencoder.layers[-1]
decoder_model = Model(inputs=encoded_input, outputs=decoder_layer(encoded_input))

# Decode the encoded images
decoded_imgs = decoder_model.predict(encoded_imgs)


11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/50
938/938 ━━━━━━━━━━━━━━━━━━━━ 8s 8ms/step - loss: 0.0672 - val_loss: 0.0228
Epoch 2/50
938/938 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - loss: 0.0203 - val_loss: 0.0140
Epoch 3/50
938/938 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - loss: 0.0135 - val_loss: 0.0114
Epoch 4/50
938/938 ━━━━━━━━━━━━━━━━━━━━ 6s 6ms/step - loss: 0.0114 - val_loss: 0.0106
Epoch 5/50
938/938 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - loss: 0.0108 - val_loss: 0.0103
Epoch 6/50
938/938 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - loss: 0.0106 - val_loss: 0.0102
Epoch 7/50
938/938 ━━━━━━━━━━━━━━━━━━━━ 6s 6ms/step - loss: 0.0104 - val_loss: 0.0101
Epoch 8/50
938/938 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - loss: 0.0104 - val_loss: 0.0100
Epoch 9/50
938/938 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - loss: 0.0103 - val_loss: 0.0100
Epoch 10/50
938/938 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - loss: 0.0102 - val_loss: 0.0099
Epoch 11/50
938/938 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - loss: 0.0102 - val_loss: 0.0099
E

In [3]:
import numpy as np
import matplotlib.pyplot as plt
from keras.models import Model
from keras.layers import Input, Dense
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Generate synthetic data
normal_data = np.random.normal(loc=0.0, scale=1.0, size=(1000, 10))
anomalies = np.random.uniform(low=-5.0, high=5.0, size=(50, 10))

# Scale and label the data
scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(np.concatenate((normal_data, anomalies), axis=0))
labels = np.concatenate((np.zeros(normal_data.shape[0]), np.ones(anomalies.shape[0])), axis=0)

scaler = StandardScaler()
data_scaled = scaler.fit_transform(data_scaled)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(data_scaled, labels, test_size=0.2, random_state=42)

# Train the autoencoder on normal data only
X_train = X_train[y_train == 0]

# Define the autoencoder model
input_dim = X_train.shape[1]
input_layer = Input(shape=(input_dim,))
encoded = Dense(6, activation='relu')(input_layer)
encoded = Dense(4, activation='relu')(encoded)
decoded = Dense(6, activation='relu')(encoded)
decoded = Dense(input_dim, activation='linear')(decoded)
autoencoder = Model(inputs=input_layer, outputs=decoded)
autoencoder.compile(optimizer='adam', loss='mse')

# Train the autoencoder
autoencoder.fit(X_train, X_train, epochs=50, batch_size=32, shuffle=True, validation_split=0.1)

# Predict on the test set and calculate the reconstruction error
X_test_pred = autoencoder.predict(X_test)
mse = np.mean(np.power(X_test - X_test_pred, 2), axis=1)

# Determine the threshold for anomaly detection
threshold = np.percentile(mse, 95)
print("Reconstruction Error Threshold:", threshold)
y_pred = (mse > threshold).astype(int)

from sklearn.metrics import confusion_matrix, classification_report
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

Epoch 1/50
23/23 ━━━━━━━━━━━━━━━━━━━━ 6s 26ms/step - loss: 0.7959 - val_loss: 0.7079
Epoch 2/50
23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.7763 - val_loss: 0.6977
Epoch 3/50
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.7534 - val_loss: 0.6900
Epoch 4/50
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.7260 - val_loss: 0.6827
Epoch 5/50
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.7173 - val_loss: 0.6756
Epoch 6/50
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.7080 - val_loss: 0.6688
Epoch 7/50
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.6959 - val_loss: 0.6621
Epoch 8/50
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.6813 - val_loss: 0.6540
Epoch 9/50
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.6935 - val_loss: 0.6448
Epoch 10/50
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.6786 - val_loss: 0.6353
Epoch 11/50
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.6825 - val_loss: 0.6237
Epoch 12/50
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.6497 - val_l